In [ ]:

import torch
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd 
import torch 
from torch.utils.data import Dataset, DataLoader 
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from tqdm import tqdm

# Load your data 
# df = pd.read_csv("final_processed.csv") 
df = pd.read_csv("sampled_data.csv") 

# Drop rows with NaN values
df = df.dropna()

# Display the DataFrame to verify the data 
print(df.head()) 

# Map labels to integers 
label_to_int = {label: i for i, label in enumerate(df['sub_category'].unique())} 
df['sub_category'] = df['sub_category'].map(label_to_int) 

# Load the trained model and tokenizer
model = BertForSequenceClassification.from_pretrained("trained_bert_model")
tokenizer = BertTokenizer.from_pretrained("trained_bert_tokenizer")

# Ensure model is in evaluation mode
model.eval()


int_to_label = {v: k for k, v in label_to_int.items()}

# Define the prediction function
def predict(model, tokenizer, texts, max_length=128):
    model.eval()
    predictions = []
    with torch.no_grad():
        for text in texts:
            # Tokenize the input text
            tokens = tokenizer(
                text,
                padding='max_length',
                max_length=max_length,
                truncation=True,
                return_tensors="pt"
            )
            tokens = {key: val.to(model.device) for key, val in tokens.items()}
            
            # Get model outputs
            output = model(**tokens)
            _, predicted_class = torch.max(output.logits, dim=1)
            predictions.append(predicted_class.item())
    return predictions

# Input texts to classify
unknown_texts = [
    "received URL link for updating KYC from mobile no after opening received otp and entered and confirmation received saying KYC done like this received OTP till KYC after that started receiving sms from bank after sec delay on amount cut and then stopped further otp Bank is also responsible for delaying sms",
    "some unknown user has send an email to my rwa and making my reputation at stake along with my family . i request you to please take a legal action and help me to find the person who has is ruing my personal reputation ."
]

# Perform prediction
predicted_classes = predict(model, tokenizer, unknown_texts)

# Map predictions back to label names
for text, prediction in zip(unknown_texts, predicted_classes):
    predicted_label = int_to_label[prediction]
    print(f"Text: {text}\nPredicted Class: {prediction} | Class Label: {predicted_label}\n")
